# cesm Rainfall Anomalies

In [ ]:
#import my functions
import helpers.fileHandler as fh
import utils._modelDefinitions as _model
import utils._indexDefinitions as _index
import utils.timePeriod as tp

In [ ]:
import xarray
import numpy
import cftime
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.offsetbox import AnchoredText
import cartopy.io.shapereader as shpr

%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
secondsPerDay = 60*60*24
secondsToTimeP = secondsPerDay*1000 #seconds per day, convert m to mm
#secondsToTimeP = secondsPerDay*_index.nDaysOfInterest["precAnom"]*1000/7 #seconds per month

# Carve out an area of interest, and calculate anomalies

In [ ]:
_model.scenarioMip[:,1]

In [ ]:
#model='ACCESS-CM2'
for model in _model.scenarioMip[18:]:
    #Calculate a climatology
    #Based on the control run, calculate monthly anomalies
    print(model)
    xr = xarray.merge([
        fh.loadModelData(model[1], 'pr_Amon', 'piControl', model[2]),
        fh.loadModelData(model[1], 'tas_Amon', 'piControl', model[2])
    ], compat='override')
    monMeansDa=xr.groupby('time.month').mean(dim='time')
    monMeansDa.where(
            (monMeansDa.lat>-50) & (monMeansDa.lat<0) & (monMeansDa.lon>100) & (monMeansDa.lon<170),
            drop=True
        ).load()
    monMeansDa.to_netcdf('results/cmipMonthlyPrTs/monMeans'+model[1]+experiment.capitalize()+'.nc')

    
    #anom for piControl
    anomDa=xr.groupby('time.month')-monMeansDa
    anomDa['model']=model[1]

    #grab area around Australia
    domainDa=anomDa.where(
        (anomDa.lat>-50) & (anomDa.lat<0) & (anomDa.lon>100) & (anomDa.lon<170),
        drop=True
    )

    domainDa.to_netcdf(
        'results/cmipMonthlyPrTs/'+model[1]+'PiControl.nc')
    domainDa.close()
    
    monMeansDa=xarray.open_Dataset(
        'results/cmipMonthlyPrTs/monMeans'+model[1]+experiment.capitalize()+'.nc')
    
    #calculate anomalies for all scenarios
    for experiment in _model.scenarioSet:
        
        try: 
            #load it
            xr = xarray.merge([
                fh.loadModelData(model[1], 'pr_Amon', experiment, model[3]),
                fh.loadModelData(model[1], 'tas_Amon', experiment, model[3])
            ], compat='override')

            #anom
            anomDa=xr.groupby('time.month')-monMeansDa
            anomDa['model']=model[1]

            #grab area around Australia
            domainDa=anomDa.where(
                (anomDa.lat>-50) & (anomDa.lat<0) & (anomDa.lon>100) & (anomDa.lon<170),
                drop=True
            )

            domainDa.to_netcdf(
                'results/cmipMonthlyPrTs/'+model[1]+'_'+experiment+'.nc')
        except Exception as e:
            print(model[1] + " did not calculate")
            print(e)

# Time averaging for the anomalies

In [ ]:
for experiment in _model.experimentSet:
    monthlyAnomDa=xarray.concat([
        xarray.open_dataset(
            'results/cmipMonthlyPrTs/'+model+'_'+experiment+'.nc') for model in _model.scenarioMip],
    'model')
    

    monthlyAnomDa=monthlyAnomDa.assign({'model':_model.scenarioMip})
    warmSeasonAnomDa=xarray.DataArray()
    
    warmSeasonAnomDa['pr']=secondsToTimeP*tp.averageForTimePeriod(monthlyAnomDa.pr)
    warmSeasonAnomDa['ts']=tp.averageForTimePeriod(monthlyAnomDa.ts)
    warmSeasonAnomDa = warmSeasonAnomDa.assign_attrs([*warmSeasonAnomDa.attrs, ('units','mm/month'), ('timePeriod','Warm Season')])

    warmSeasonAnomDa.to_netcdf('results/cmip'+experiment.capitalize()+'PrecAnoms.nc')